# Лабораторная работа №5
## Выполнил студент группы Возжин Сергей Дмитриевич БПИ2301

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [ ]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [2]:
def kmp_search(text, pattern, case_sensitive=True):
    """
    Поиск подстроки в строке с использованием алгоритма Кнута-Морриса-Пратта (KMP).

    :param text: Строка, в которой выполняется поиск
    :param pattern: Искомая подстрока
    :param case_sensitive: Флаг чувствительности к регистру
    :return: Индекс начала подстроки или -1, если не найдено
    """
    if not case_sensitive:
        text = text.lower()
        pattern = pattern.lower()

    p = [0] * len(pattern)
    j = 0
    for i in range(1, len(pattern)):
        while j > 0 and pattern[i] != pattern[j]:
            j = p[j - 1]
        if pattern[i] == pattern[j]:
            j += 1
            p[i] = j

    i = j = 0
    while i < len(text):
        if text[i] == pattern[j]:
            i += 1
            j += 1
            if j == len(pattern):
                return i - j
        else:
            if j > 0:
                j = p[j - 1]
            else:
                i += 1

    return -1

text = "lirili larilalalala"
pattern = "laLa"

print(kmp_search(text, pattern, case_sensitive=True))
print(kmp_search(text, pattern, case_sensitive=False))

-1
11


 #### Упрощенный алгоритм Бойера-Мура

In [5]:
def boyer_moore_search(text, pattern, case_sensitive=True):
    """
    Поиск подстроки в строке с использованием алгоритма Бойера-Мура.

    :param text: Строка, в которой выполняется поиск
    :param pattern: Искомая подстрока
    :param case_sensitive: Флаг чувствительности к регистру
    :return: Индекс начала подстроки или -1, если не найдено
    """
    if not case_sensitive:
        text = text.lower()
        pattern = pattern.lower()
    S = set()
    M = len(pattern)
    d = {}

    for i in range(M - 2, -1, -1):
        if pattern[i] not in S:
            d[pattern[i]] = M - i - 1
            S.add(pattern[i])

    if pattern[M - 1] not in S:
        d[pattern[M - 1]] = M
        S.add(pattern[M - 1])

    d["*"] = M

    print("Таблица смещений:", d)

    N = len(text)

    if N >= M:
        i = M - 1
        while i < N:
            k = 0
            for j in range(M - 1, -1, -1):
                if text[i - k] != pattern[j]:
                    off = d.get(text[i], d["*"])
                    i += off
                    break
                k += 1

            if k == M:
                return i - k + 1
        else:
            return -1
    else:
        return -1

text = "тралалеро тралала"
pattern = "лаЛа"

print(boyer_moore_search(text, pattern, case_sensitive=True))
print(boyer_moore_search(text, pattern, case_sensitive=False))

Таблица смещений: {'Л': 1, 'а': 2, 'л': 3, '*': 4}
-1
Таблица смещений: {'л': 1, 'а': 2, '*': 4}
13


In [12]:
import time
def measure_time(func, text, pattern, case_sensitive):
    start = time.time()
    result = func(text, pattern, case_sensitive)
    end = time.time()
    return result, (end - start) * 1000  # Время в миллисекундах


def main():
    # Тестовые данные
    text = "тралалеро тралала"
    pattern = "лала"
    case_sensitive = False

    print("\n=== Результаты поиска ===")
    print(f"Текст: {text}")
    print(f"Подстрока: {pattern}")
    print(f"Чувствительность к регистру: {'Да' if case_sensitive else 'Нет'}\n")

    # Алгоритм KMP
    result, kmp_time = measure_time(kmp_search, text, pattern, case_sensitive)
    print(f"KMP: {'Найдено на позиции ' + str(result) if result != -1 else 'Не найдено'} за {kmp_time:.4f} мс")

    # Алгоритм Бойера-Мура
    result, bm_time = measure_time(boyer_moore_search, text, pattern, case_sensitive)
    print(f"Boyer-Moore: {'Найдено на позиции ' + str(result) if result != -1 else 'Не найдено'} за {bm_time:.4f} мс")

    # Встроенный поиск
    start = time.time()
    if case_sensitive:
        result = text.find(pattern)
    else:
        result = text.lower().find(pattern.lower())
    end = time.time()
    builtin_time = (end - start) * 1000

    print(f"Built-in find(): {'Найдено на позиции ' + str(result) if result != -1 else 'Не найдено'} за {builtin_time:.4f} мс")

    # Сравнение времени выполнения
    print("\n=== Сравнение времени выполнения ===")
    print(f"KMP: {kmp_time:.4f} мс")
    print(f"Boyer-Moore: {bm_time:.4f} мс")
    print(f"Built-in find(): {builtin_time:.4f} мс")


if __name__ == "__main__":
    main()


=== Результаты поиска ===
Текст: тралалеро тралала
Подстрока: лала
Чувствительность к регистру: Нет

KMP: Найдено на позиции 13 за 0.0050 мс
Таблица смещений: {'л': 1, 'а': 2, '*': 4}
Boyer-Moore: Найдено на позиции 13 за 0.0122 мс
Built-in find(): Найдено на позиции 13 за 0.0010 мс

=== Сравнение времени выполнения ===
KMP: 0.0050 мс
Boyer-Moore: 0.0122 мс
Built-in find(): 0.0010 мс


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [13]:
from collections import deque


def is_solvable(puzzle, n):
    inversions = 0
    empty_row = 0

    for i in range(n*n):
        if puzzle[i] == 0:
            empty_row = i // n
            continue
        for j in range(i+1, n*n):
            if puzzle[j] != 0 and puzzle[i] > puzzle[j]:
                inversions += 1

    if n % 2 == 1:
        return inversions % 2 == 0
    else:
        return (inversions + empty_row) % 2 == 1

def generate_goal_state(n):
    return list(range(1, n * n)) + [0]

def get_neighbors(state, n):
    neighbors = []
    empty_index = state.index(0)
    row, col = divmod(empty_index, n)

    for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < n and 0 <= new_col < n:
            new_index = new_row * n + new_col
            new_state = state[:]
            new_state[empty_index], new_state[new_index] = new_state[new_index], new_state[empty_index]
            neighbors.append((new_state, new_state[empty_index]))

    return neighbors

def solve_puzzle(start_state, n):
    if not is_solvable(start_state, n):
        return []

    goal_state = generate_goal_state(n)

    if start_state == goal_state:
        return []

    queue = deque([(start_state, [])])
    visited = set([tuple(start_state)])

    while queue:
        current_state, path = queue.popleft()

        for neighbor, moved_tile in get_neighbors(current_state, n):
            if tuple(neighbor) not in visited:
                if neighbor == goal_state:
                    return path + [moved_tile]

                visited.add(tuple(neighbor))
                queue.append((neighbor, path + [moved_tile]))

    return []

puzzle = [1, 2, 3, 4,
          5, 6, 7, 8,
          9, 14, 10, 12,
          0, 13, 11, 15]

solution = solve_puzzle(puzzle, 4)
print("Решение найдено:", solution)

Решение найдено: [13, 14, 10, 11, 15]


### Вывод